<a href="https://colab.research.google.com/github/musicjae/korean_sentiment_analysis/blob/master/bert_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.9MB 5.6MB/s 
     |████████████████████████████████| 3.2MB 24.2MB/s 
     |████████████████████████████████| 890kB 40.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=b3fef072670c0c858cf985da66ab3e2911a9da49202c40b017a3815e7b522bb4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 18.79 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

# Args

In [8]:
import argparse

args = argparse.ArgumentParser()

args.pretrain_model = "beomi/kcbert-base"

# Data Load

In [4]:
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


In [5]:
sentences = train['document']
sentences[:10]

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
5        막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
6                                원작의 긴장감을 제대로 살려내지못했다.
7    별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...
8                               액션이 없는데도 재미 있는 몇안되는 영화
9        왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?
Name: document, dtype: object

# Preprocessing

### Tokenizer

- 버트는 모든 데이터 내의 문장들에 [CLS], [SEP] 패딩을 취한다. 이 전처리를 수행하기 위해 아래 같은 과정을 거친다.

In [6]:
sentences = ["[CLS]"+str(sentence) + "[SEP]" for sentence in sentences]
sentences[:3]

['[CLS]아 더빙.. 진짜 짜증나네요 목소리[SEP]',
 '[CLS]흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나[SEP]',
 '[CLS]너무재밓었다그래서보는것을추천한다[SEP]']

- 토큰화를 수행하기 위해, 미리 학습된 kcbert를 불러온 뒤 이것을 이용하여 토크나이저를 만든다.

- 데이터 내의 각각의 모든 문장에 대해 토큰화 수행

In [11]:
tokenizer = BertTokenizer.from_pretrained(args.pretrain_model, do_lower_case=False)
tok_texts = [tokenizer.tokenize(sent) for sent in sentences]

print(tok_texts[:3])

[['[CLS]', '아', '더', '##빙', '.', '.', '진짜', '짜증나네', '##요', '목소리', '[SEP]'], ['[CLS]', '흠', '.', '.', '.', '포', '##스터', '##보고', '초딩', '##영화', '##줄', '.', '.', '.', '.', '오버', '##연기', '##조차', '가볍', '##지', '않', '##구나', '[SEP]'], ['[CLS]', '너무', '##재', '##밓', '##었다', '##그래', '##서', '##보는', '##것을', '##추', '##천', '##한다', '[SEP]']]


### Padding

In [ ]:
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tok_texts] # 텍스트 정수 인코딩
input_ids = pad_sequences(input_ids)
